This notebook is solely used for Coursera Data Capstone Project

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Capstone project - Clustering: Part 1

Import the necessary libraries

In [2]:
import requests
import bs4

Extract the table from the wikipedia page

In [3]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = bs4.BeautifulSoup(response.text, 'html.parser')
table = html.find('table',{'class':'wikitable sortable'})

Iterate over the data in the table, getting all 3 columns and cleaning the data

In [4]:
location_data = []
for i in table.findAll('tr'):
    row_data=[]
    for j in i.findAll('td'):
        row_data.append(j.text.strip())
    location_data.append(row_data)

Create a pandas dataframe with the proper column names. Clean up the unassigned boroughs. There are no unassigned neighbourhoods that have an assigned borough, so we don't need to rename those. Duplicate postal codes have already been merged in the wiki table so we don't have to do that anymore either.

In [5]:
df = pd.DataFrame(location_data, columns = ['Postalcode','Borough','Neighbourhood'])
df.dropna(inplace=True)
df = df[df['Borough']!='Not assigned']
df.drop_duplicates(subset =['Borough','Neighbourhood'])
df.reset_index(inplace=True,drop=True)

In [6]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
df.shape

(103, 3)

# Capstone Project - Clustering: Part 2

Importing geopy library modules to retrieve location data

In [8]:
from geopy.geocoders import Nominatim
geo_locator = Nominatim(user_agent ='toronto_explorer')

In [9]:
 locations = []
for neighbourhood in df['Neighbourhood']:
    locations.append(geo_locator.geocode(neighbourhood.split(',')[0].split('-')[0]+' Toronto Canada'))

In [10]:
df['locations'] = locations
df.dropna(inplace=True)


In [11]:
longitudes = []
latitudes = []
for location in df['locations']:
    longitudes.append(location.longitude)
    latitudes.append(location.latitude)

In [12]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes
df.drop(['locations'],axis=1,inplace=True)

In [13]:
df.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.659659,-79.390340


# Capstone Project - Clustering: Part 3

In [14]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

We clone the dataset here so we can easily change it later and keep the same code to analyse it.

In [15]:
#toronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data = df


Foursquare client details

In [16]:
CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version


Define a function go search FourSquare for nearby venues in each neighbourhood, and extrac the data

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

One hot encoding the category data so it is ready for analysis.

In [35]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()

Here we look at the top 5 venue category in each neighbourhood

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0           Food Court   2.0
1   Chinese Restaurant   2.0
2  Rental Car Location   2.0
3     Asian Restaurant   1.0
4        Shopping Mall   1.0


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   2.0
1  Sandwich Place   1.0
2            Pool   1.0
3             Gym   1.0
4             Pub   1.0


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                 Playground   1.0
1                       Park   1.0
2             Baseball Field   1.0
3          Convenience Store   1.0
4  Middle Eastern Restaurant   0.0


----Bayview Village----
                  venue  freq
0                  Bank   2.0
1        Breakfast Spot   1.0
2    Persian Restaurant   1.0
3  Outdoor Supply Store   1.0
4        Clothing Store   1.0


----Bedford Park, Lawrence Manor East----
                        venue  freq
0  Construction & Landscaping   1.0
1                 Yoga Studio   0.0
2    

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


In [40]:
from sklearn.cluster import KMeans
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)

In [41]:
kmeans = KMeans(n_clusters=kclusters).fit(toronto_grouped_clustering)

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.dropna(inplace=True)
toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.758800,-79.320197,1.0,Pizza Place,Bank,Convenience Store,Coffee Shop,Discount Store,Chinese Restaurant,Caribbean Restaurant,Electronics Store,Shopping Mall,Liquor Store
1,M4A,North York,Victoria Village,43.732658,-79.311189,1.0,Mediterranean Restaurant,Spa,Thai Restaurant,Park,Middle Eastern Restaurant,Women's Store,Eastern European Restaurant,Doctor's Office,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457,1.0,Coffee Shop,Thai Restaurant,Performing Arts Venue,Pool,Fast Food Restaurant,Park,Food Truck,Beer Store,Restaurant,Sushi Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507,1.0,Electronics Store,Park,Kids Store,Doctor's Office,Bank,Falafel Restaurant,Event Space,Farm,Ethiopian Restaurant,Dive Bar
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.659659,-79.390340,2.0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Japanese Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Bubble Tea Shop,Mediterranean Restaurant


In [43]:
address = 'Toronto Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [44]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to map
for lat, lng, borough, neighborhood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,North York,0.0,Clothing Store,Coffee Shop,Japanese Restaurant,Restaurant,Fast Food Restaurant,Bank,Bakery,Food Court,Sandwich Place,Convenience Store
8,East York,0.0,Coffee Shop,Clothing Store,Hotel,Cosmetics Shop,Japanese Restaurant,Restaurant,Plaza,Steakhouse,Diner,Thai Restaurant
9,Downtown Toronto,0.0,Clothing Store,Coffee Shop,Hotel,Theater,Bookstore,Japanese Restaurant,Cosmetics Shop,Lingerie Store,Sandwich Place,Restaurant
13,North York,0.0,Clothing Store,Coffee Shop,Japanese Restaurant,Restaurant,Fast Food Restaurant,Bank,Bakery,Food Court,Sandwich Place,Convenience Store
24,Downtown Toronto,0.0,Clothing Store,Coffee Shop,Sushi Restaurant,Boutique,Italian Restaurant,French Restaurant,Mediterranean Restaurant,Japanese Restaurant,Café,Smoke Shop


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Pizza Place,Bank,Convenience Store,Coffee Shop,Discount Store,Chinese Restaurant,Caribbean Restaurant,Electronics Store,Shopping Mall,Liquor Store
1,North York,1.0,Mediterranean Restaurant,Spa,Thai Restaurant,Park,Middle Eastern Restaurant,Women's Store,Eastern European Restaurant,Doctor's Office,Dog Run,Doner Restaurant
2,Downtown Toronto,1.0,Coffee Shop,Thai Restaurant,Performing Arts Venue,Pool,Fast Food Restaurant,Park,Food Truck,Beer Store,Restaurant,Sushi Restaurant
3,North York,1.0,Electronics Store,Park,Kids Store,Doctor's Office,Bank,Falafel Restaurant,Event Space,Farm,Ethiopian Restaurant,Dive Bar
5,Etobicoke,1.0,Smoothie Shop,Sandwich Place,Food Service,Coffee Shop,Women's Store,Eastern European Restaurant,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2.0,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Japanese Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Bubble Tea Shop,Mediterranean Restaurant
20,Downtown Toronto,2.0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Hotel,Gym,Seafood Restaurant,Bakery,Gastropub
36,Downtown Toronto,2.0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Bank,Sporting Goods Shop,Gym,Steakhouse
42,Downtown Toronto,2.0,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Japanese Restaurant,Seafood Restaurant,Sushi Restaurant,Italian Restaurant,Salad Place
48,Downtown Toronto,2.0,Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant,Gym,American Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,3.0,Coffee Shop,Pizza Place,Café,Grocery Store,Bakery,Bistro,Bar,Beer Store,Restaurant,Hotel
25,Downtown Toronto,3.0,Korean Restaurant,Coffee Shop,Indian Restaurant,Mexican Restaurant,Karaoke Bar,Dessert Shop,Café,Cocktail Bar,Ice Cream Shop,Grocery Store
52,North York,3.0,Coffee Shop,Japanese Restaurant,Grocery Store,Bank,Pharmacy,Burger Joint,Sandwich Place,Gas Station,Fried Chicken Joint,Korean Restaurant
59,North York,3.0,Coffee Shop,Japanese Restaurant,Grocery Store,Bank,Pharmacy,Burger Joint,Sandwich Place,Gas Station,Fried Chicken Joint,Korean Restaurant
61,Central Toronto,3.0,Sushi Restaurant,Coffee Shop,Italian Restaurant,Bakery,Pub,Bank,Fast Food Restaurant,Asian Restaurant,Cosmetics Shop,Café


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head()

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,4.0,Bar,Bakery,Café,Coffee Shop,Beer Store,Mexican Restaurant,Sandwich Place,Cocktail Bar,Restaurant,Gift Shop
37,West Toronto,4.0,Bar,Café,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Korean Restaurant,French Restaurant,Thai Restaurant,Dive Bar
73,Central Toronto,4.0,Bar,Vegetarian / Vegan Restaurant,Café,Asian Restaurant,Restaurant,Vietnamese Restaurant,Men's Store,Coffee Shop,Theater,French Restaurant
84,Downtown Toronto,4.0,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Bakery,Dessert Shop,Vietnamese Restaurant,Nightclub,Middle Eastern Restaurant
94,Etobicoke,4.0,Bar,Vegetarian / Vegan Restaurant,Café,Asian Restaurant,Restaurant,Vietnamese Restaurant,Men's Store,Coffee Shop,Theater,French Restaurant
